## What are sockets?

- we will stick to TCP (or STREAM) sockets and to INET (or IPV4) sockets
- a client program only has a client socket
- a server program has both client and server sockets: the server socket is the thing that sits and listens for connections, spawning a client socket to deal with them.

### Why do we care?

- we'll need it for the client communications for our database
- you will want to know how to use them to make custom data servers and clients. We provide here an example of a no-copy numpy array sharing over tcp sockets.

### A simple server and its client

In [77]:
%%file server0.py
from socket import *
s = socket(AF_INET, SOCK_STREAM)
s.bind(('', 25000))
s.listen(1)
c,a = s.accept()

Overwriting server0.py


- `s` here is a server socket. It binds to '', or any address on this machine, and port 25000. 
- Low number ports are reserved by system services, only root can create them.
- listen tells the socker to queue in only `1` request here before refusing outside connections. 5-10 is plenty, if your code is well written
- `accept` creates a client socket `c` with address `a`

In [10]:
%%file client0.py
from socket import *
c = socket(AF_INET, SOCK_STREAM)
c.connect(('localhost', 25000))

Overwriting client0.py


From the client:

- when you are `connect`ed the socket `c` can be used to send in a request or to recieve some data. 
- This socket will read the response and then be destroyed. Client sockets are one-shot
- the client's client socket and server's client socket are the same type of thing, so you need to decide "who calls whom"

Communication happens using some variant of `send` and `recv`, or `read` and `write` as we saw in the crawler earlier when we had given a file like object to our sockets. As with the latter, for the former it is your responsibility to call and call again until the buffers are full. We do that below, with the slight difference that we are reading into already allocated buffers.

### Sharing data with sockets

In [9]:
%%file client0.py
import numpy as np
from socket import *


def recv_into(arr, source):
    view = memoryview(arr).cast('B') 
    while len(view):
        nrecv = source.recv_into(view)
        print("recieved", nrecv)
        view = view[nrecv:]
            
c = socket(AF_INET, SOCK_STREAM)
c.connect(('localhost', 25000))
a = np.zeros(shape=50000000, dtype=float)
print(a[0:10])
recv_into(a, c)
print(a[0:10])

Overwriting client0.py


In [8]:
%%file server0.py
from socket import *
import numpy as np

def send_from(arr, dest):
    view = memoryview(arr).cast('B') 
    while len(view):
        nsent = dest.send(view)
        view = view[nsent:]
            

s = socket(AF_INET, SOCK_STREAM)
s.bind(('', 25000))
s.listen(1)
c,addr = s.accept()
a = np.arange(0.0, 50000000.0)
send_from(a, c)

Overwriting server0.py


In this case we have choden to have the server "write-to" the client. We could have had the client request the server. We'll do that next

Notice that so far we have not tried to keep the server socket persistent. We'll do this next as well. But first let us write the cacnonical echo server. We can test this with `telnet`.

### The canonical echo server

In [42]:
%%file echo_server.py
from socket import socket, AF_INET, SOCK_STREAM

def echo_handler(address, client_sock):
    print('Got connection from {}'.format(address)) 
    while True:
        msg = client_sock.recv(2)
        print(len(msg))
        if not msg:
            print("broke")
            break
        client_sock.sendall(msg+b"||||")
    client_sock.close()

def echo_server(address, backlog=5): 
    sock = socket(AF_INET, SOCK_STREAM) 
    sock.bind(address) 
    sock.listen(backlog)
    while True:
        client_sock, client_addr = sock.accept() 
        echo_handler(client_addr, client_sock)
        
if __name__ == '__main__': 
    echo_server(('', 20001))

Overwriting echo_server.py


### Making the server persistent

In [51]:
%%file server1.py
from socket import *
import numpy as np

a = np.arange(0.0, 50000000.0)

def send_from(arr, dest):
    view = memoryview(arr).cast('B') 
    while len(view):
        nsent = dest.send(view)
        view = view[nsent:]
            
def handle(csock):
    msg = b""
    while True:
        recvd = csock.recv(2)#chosen to be large enough
        print("recieved", recvd)
        if not recvd: # handle close
            break
        msg += recvd
        if len(msg)==10:
            break
    offset, numele=msg.decode().split(':')
    offset=int(offset)
    numele=int(numele)
    send_from(a[offset:offset+numele], csock)
    csock.close()
        
def array_server(address_tuple, backlog=1):
    s = socket(AF_INET, SOCK_STREAM)
    s.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
    s.bind(address_tuple)
    s.listen(backlog)
    
    while True:
        csock,caddr = s.accept()
        print("got connection from {}".format(caddr))
        handle(csock)
    s.close()

array_server(('', 25000))

Overwriting server1.py


In [52]:
%%file client1.py
import numpy as np
from socket import *
offset=5000
num_wanted=10000

def recv_into(arr, source):
    view = memoryview(arr).cast('B') 
    while len(view):
        nrecv = source.recv_into(view)
        print("recieved", nrecv)
        view = view[nrecv:]
            

c = socket(AF_INET, SOCK_STREAM)
c.connect(('localhost', 25000))
mybytes=str.encode("{}:{}".format(offset, num_wanted))
print(len(mybytes))
c.send(mybytes)
a = np.zeros(shape=num_wanted, dtype=float)
print(a[0:10])
recv_into(a, c)
print(a[0:10])

Overwriting client1.py


## Using the socket server module

In [57]:
%%file echo_server2.py
from socketserver import BaseRequestHandler, TCPServer

class EchoHandler(BaseRequestHandler): 
    def handle(self):
        print('Got connection from', self.client_address) 
        while True:
            msg = self.request.recv(8192)
            if not msg:
                break
            self.request.send(msg)
            
if __name__ == '__main__':
    serv = TCPServer(('', 20000), EchoHandler) 
    serv.serve_forever()

Writing echo_server2.py


In [58]:
%%file echo_client.py
import sys
from socket import socket, AF_INET, SOCK_STREAM
s = socket(AF_INET, SOCK_STREAM)
s.connect(('localhost', 20000))
s.send(sys.argv[1].encode())
print(s.recv(8192))

Overwriting echo_client.py


### Threaded implementations

In [59]:
%%file echo_server3.py
from socketserver import BaseRequestHandler, ThreadingTCPServer

class EchoHandler(BaseRequestHandler): 
    def handle(self):
        print('Got connection from', self.client_address) 
        while True:
            msg = self.request.recv(8192)
            if not msg:
                break
            self.request.send(msg)
            
if __name__ == '__main__':
    serv = ThreadingTCPServer(('', 20000), EchoHandler) 
    serv.serve_forever()

Writing echo_server3.py


In [62]:
%%file echo_server4.py
from socketserver import BaseRequestHandler, TCPServer

class EchoHandler(BaseRequestHandler): 
    def handle(self):
        print('Got connection from', self.client_address) 
        while True:
            msg = self.request.recv(8192)
            if not msg:
                break
            self.request.send(msg)

if __name__ == '__main__':
    from threading import Thread
    NWORKERS = 16
    TCPServer.allow_reuse_address = True
    serv = TCPServer(('', 20000), EchoHandler) 
    
    for n in range(NWORKERS):
            t = Thread(target=serv.serve_forever)
            t.daemon = True
            t.start()
    serv.serve_forever()

Overwriting echo_server4.py


## Asyncio

### The Streams way

This is the higher level api.

In [73]:
%%file echo_server_streams_asyncio.py

import asyncio

#reader is of type StreamReader, writer of type StreamWriter
async def handle_echo(reader, writer):
    #coroutine reads data from client
    data = await reader.read(8192)
    message = data.decode()
    addr = writer.get_extra_info('peername')
    print("Received %r from %r" % (message, addr))

    print("Send: %r" % message)
    writer.write(data) # NOT a coroutine
    await writer.drain() #coroutine flushes write buffer

    print("Close the client socket")
    writer.close()

loop = asyncio.get_event_loop()
coro = asyncio.start_server(handle_echo, '', 20000, loop=loop)

#drive server coroutine
server = loop.run_until_complete(coro)

# Serve requests until Ctrl+C is pressed: this runs event loop
print('Serving on {}'.format(server.sockets[0].getsockname()))
try:
    loop.run_forever()
except KeyboardInterrupt:
    pass

# Close the server
server.close()
#inside the brackets is a future. The loop waits until all is closed.
loop.run_until_complete(server.wait_closed())
loop.close()

Overwriting echo_server_streams_asyncio.py


In [74]:
%%file echo_client_streams_asyncio.py
import asyncio
import sys

async def tcp_echo_client(message, loop):
    reader, writer = await asyncio.open_connection('', 20000,
                                                        loop=loop)

    print('Send: %r' % message)
    writer.write(message.encode())

    data = await reader.read(8192)
    print('Received: %r' % data.decode())

    print('Close the socket')
    writer.close()

message = sys.argv[1]
loop = asyncio.get_event_loop()
loop.run_until_complete(tcp_echo_client(message, loop))
loop.close()

Overwriting echo_client_streams_asyncio.py


### The protocols way

This is the (lower level) callbacks based api that the asyncip library exposes.

In [76]:
%%file echo_server_protocols_asyncio.py

import asyncio

class EchoServerClientProtocol(asyncio.Protocol):
    def connection_made(self, transport):
        peername = transport.get_extra_info('peername')
        print('Connection from {}'.format(peername))
        self.transport = transport

    def data_received(self, data):
        message = data.decode()
        print('Data received: {!r}'.format(message))

        print('Send: {!r}'.format(message))
        self.transport.write(data)

        print('Close the client socket')
        self.transport.close()

loop = asyncio.get_event_loop()
# Each client connection will create a new protocol instance
coro = loop.create_server(EchoServerClientProtocol, '', 20000)
server = loop.run_until_complete(coro)

# Serve requests until Ctrl+C is pressed
print('Serving on {}'.format(server.sockets[0].getsockname()))
try:
    loop.run_forever()
except KeyboardInterrupt:
    pass

# Close the server
server.close()
loop.run_until_complete(server.wait_closed())
loop.close()


Writing echo_server_protocols_asyncio.py


From docs:
>Transport.close() can be called immediately after WriteTransport.write() even if data are not sent yet on the socket: both methods are asynchronous. yield from is not needed because these transport methods are not coroutines.

In [75]:
%%file echo_client_protocols_asyncio.py

import asyncio
import sys

class EchoClientProtocol(asyncio.Protocol):
    def __init__(self, message, loop):
        self.message = message
        self.loop = loop

    def connection_made(self, transport):
        transport.write(self.message.encode())
        print('Data sent: {!r}'.format(self.message))

    def data_received(self, data):
        print('Data received: {!r}'.format(data.decode()))

    def connection_lost(self, exc):
        print('The server closed the connection')
        print('Stop the event loop')
        self.loop.stop()

loop = asyncio.get_event_loop()
message = sys.argv[1]
coro = loop.create_connection(lambda: EchoClientProtocol(message, loop),
                              '127.0.0.1', 20000)
loop.run_until_complete(coro)
loop.run_forever()
loop.close()

Writing echo_client_protocols_asyncio.py


From the docs:
>The event loop is running twice. The run_until_complete() method is preferred in this short example to raise an exception if the server is not listening, instead of having to write a short coroutine to handle the exception and stop the running loop. At run_until_complete() exit, the loop is no longer running, so there is no need to stop the loop in case of an error.